# TPC6: Analisador Léxico

O trabalho para casa 6, da unidade curricular de Processamento de linguagens consiste na elaboração de analisador léxico para uma linguagem de programação previamente fornecida. Usando a biblioteca Ply o analisador léxico deve realizar a devida análise de um ficheiro de texto escrito na linguagem fornecida, e em seguida devolver a lista de tokens para a linguagem de programação.

### Exemplo 1 da linguagem fornecida

```
/* factorial.p
-- 2023-03-20 
-- by jcr
*/

int i;

// Função que calcula o factorial dum número n

function fact(n){
  int res = 1;
  while res > 1 {
    res = res * n;
    res = res - 1;
  }
}

// Programa principal
program myFact{
  for i in [1..10]{
    print(i, fact(i));
  }
}

```

### Exemplo 2 da linguagem fornecida

```
/* max.p: calcula o maior inteiro duma lista desordenada
-- 2023-03-20 
-- by jcr
*/

int i = 10, a[10] = {1,2,3,4,5,6,7,8,9,10};

// Programa principal
program myMax{
  int max = a[0];
  for i in [1..9]{
    if max < a[i] {
      max = a[i];
    }
  }
  print(max);
}
```


# Bibliotecas importadas: 

In [ ]:
import ply.lex as lex
import sys

# Resolução:

In [ ]:

f = open('file.txt','r') # mudar para file2.txt para testar o segundo ficheiro

lines = f.readlines()

states = (
    ('comment', 'exclusive'),
)

tokens = ('LINECOMMENT','OPEN_BLOCKCOMMENT','CLOSE_BLOCKCOMMENT','BLOCKCOMMENT',
          'LPARENT','RPARENT','LBRACKET','RBRACKET','LSQUAREBRACKET','RSQUAREBRACKET',
          'SUM','MINUS','PRODUCT','DIVIDE','EQUAL','EQUALS','GREATER','LESS','GREATEREQUAL',
          'LESSEQUAL', 'NOTEQUALS','NUMBER','ID','NEWLINE','COMMA','SEMICOLON','DOT',
          'FOR','WHILE','FUNCTION','PROGRAM','PRINT','INT','IN','IF','ELSE'
          )


t_LPARENT = r'\('
t_RPARENT = r'\)'
t_LSQUAREBRACKET = r'\['
t_RSQUAREBRACKET = r'\]'
t_LBRACKET = r'\{'
t_RBRACKET = r'\}'
t_SUM = r'\+'
t_PRODUCT = r'\*'
t_MINUS = r'-'
t_DIVIDE = r'\/'
t_EQUAL = r'='
t_EQUALS = r'=='
t_NOTEQUALS = r'!='
t_LESS = r'<'
t_GREATER = r'>'
t_LESSEQUAL = r'<='
t_GREATEREQUAL = r'>='
t_COMMA = r','
t_SEMICOLON = r';'
t_DOT = r'\.'
t_FOR = r'(for)(?=\s?\w+)'
t_WHILE = r'(while)(?=\s?\w+)'
t_FUNCTION = r'(function)(?=\s?\w+)'
t_PROGRAM = r'(program)(?=\s?\w+)'
t_PRINT = r'(print)(?=\()'
t_IN = r'(in)(?=\s?\w+)'
t_INT = r'(int)(?=\s?\w+)'
t_IF = r'(if)(?=\s?\w+)'
t_ELSE = r'(else)'
t_LINECOMMENT = r'//[^\n]*'
t_ID = r'[a-zA-Z_]\w*'


def t_OPEN_BLOCKCOMMENT(t):
    r'^(\/\*)'
    t.lexer.begin('comment')
    t.lexer.lineno += t.value.count('\n')
    return t 

def t_comment_CLOSE_BLOCKCOMMENT(t):
    r'(\*\/)$'
    t.lexer.begin('INITIAL')
    t.lexer.lineno += t.value.count('\n')
    return t 

def t_comment_BLOCKCOMMENT(t):
    r'[\w+\.\s\t\?\(\)\[\]\{\}\+\*\/=><#$%",-:]+'
    t.lexer.lineno += t.value.count('\n')
    return t

t_comment_ignore = ' \t'

def t_comment_error(t):
    print(f"Illegal character: '{t.value[0]}', on line: {t.lexer.lineno}")
    t.lexer.skip(1)


def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

def t_NEWLINE(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

t_ignore = ' \t'

def t_error(t):
    print(f"Illegal character: '{t.value[0]}', on line: {t.lexer.lineno}")
    t.lexer.skip(1)


lexer = lex.lex()

for line in lines :
    lexer.input(line)
    while True:
        tok = lexer.token()
        if not tok: 
            break
        print(tok)


# Método de resolução:
Para a resolução deste exercicio tive como método analisar todos os tipos de tokens, após pensar quais seriam os tokens a serem capturados foi desenvolvida a lista de tokens acima. Para o caso da captura do token referente ao 'new line', é realizada uma contagem sempre que o mesmo apareça, de forma a facilitar estratégias de debug do programa. O t_error servirá para encontrar possíveis caracteres desconhecidos, e faz da contagem referente ao 'new line' para indicar o local onde o caracter desconhecido apareceu. Para o caso da captura do token referente ao comentario de múltiplas linhas tive como estratégia a criação de um estado exclusivo('comment') que encapsula-se todas as linhas referentes a este comentário, resultando em três tokens para a captura de esta situação: OPEN_BLOCKCOMMENT, CLOSE_BLOCKCOMMENT, BLOCK_COMMENT. Em todas estas situações continuo a realizar a contagem do número de linhas.

Observação: Para objetivos de teste deve correr o programa python 'compiler.py', o ficheiro jupiter apenas serve para documentação.